# Symbolic and value access
The code example below is a modification of the `Derivative` implementation that was presented in [fills](fills.ipynb). In this notebook, we will use this implementation to explore the two different ways of accessing elements. Either an access can be `symbolic`, or it can be `value`-based.

In [1]:
from openfd.alpha import Expression, GridFunction, Operator
from openfd.alpha.array import CArray

class Derivative(Operator):
    
    def initialize(self):
        self._data[0] = CArray('dl', data=[-1.0, 1.0])
        self._data[1] = CArray('di', data=[-0.5, 0.5])
        self._data[2] = CArray('dr', data=[-1.0, 1.0])
        
    def __getitem__(self, index):
        """
        This method will be moved to the base class.
        """
        return self.__eval__(index, lambda x, index : x[index])
    
    def __call__(self, index):
        """
        This method will be moved to the base class.
        """
        return self.__eval__(index, lambda x, index : x(index))
    
    def __eval__(self, index, op):
        """
        The user will only need to implement this method. 
        By using the lambda `op`, it is only necessary to write the implementation once. 
        """
        left = self.data[0]
        interior = self.data[1]
        right = self.data[2]
        
        # TODO: Build some function that handles region detection for any input (int, or symbolic)
        if isinstance(index, int):
            if index == 0:
                return op(self.args, 0)*op(left, 0) + op(self.args, 1)*op(left, 1) 
            elif index == -1:
                return op(self.args, -1)*op(right, 0) + op(self.args, -2)*op(right, 1)
            else:
                return op(self.args, index-1)*op(interior, 0) + op(self.args, index+1)*op(interior, 1)  
        else:           
            if index.id == 0:
                return op(self.args, 0)*op(left, 0) + op(self.args, 1)*op(left, 1)
            elif index.id == 2:
                return op(self.args, -1)*op(right, 0) + op(self.args, -2)*op(right, 1)
            else:
                return op(self.args, index-1)*op(interior, 0) + op(self.args, index+1)*op(interior, 1)

"""
place-holder class that will be implemented if approved. Will also include bounds as discussed in issue #13
These three classes will be one class. The region id will be an attribute.
(shameless copy-and-paste from [fills](fills.ipynb))
"""
from sympy import Symbol
class LeftBoundaryIndex(Symbol):
    @property
    def id(self):
        return 0
    
class RightBoundaryIndex(Symbol):
    @property
    def id(self):
        return 2
    
class InteriorIndex(Symbol):
    @property
    def id(self):
        return 1
    

 First, let's build an expression.

In [2]:
D = Derivative('D')
u = GridFunction('u', shape=(10,))
il = LeftBoundaryIndex('il')
ii = InteriorIndex('ii')
expr = Expression(D*u)

## Symbolic access

A symbolic access returns the evaluation of an expression in symbolic form. This means that any data values are packed into [arrays](array.ipynb). 

To perform a symbolic access of the left boundary, we pass an index that matches this region.

In [3]:
expr[il]

u[0]*dl[0] + u[1]*dl[1]

Symbolic access of the interior

In [4]:
expr[ii]

u[ii - 1]*di[0] + u[ii + 1]*di[1]

## Value access

When a value access is performed, only those objects that can return by value will do so. [Gridfunctions](gridfunction.ipynb) are by definition symbolic and will therefore not return by value. Instead, they behave in the same way as when as symbolic access occurs. However, the data [arrays](array.ipynb) that are used to build operators support value access.

Consider a value access of the left boundary

In [5]:
expr(il)

-1.0*u[0] + 1.0*u[1]

and a value access of the interior

In [6]:
expr(ii)

-0.5*u[ii - 1] + 0.5*u[ii + 1]